## Import

In [21]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [26]:
# Load historical treasury data
file_path = "./data/Anleihen und Indizes historische Zinsen.xlsx"
df_anleihen = pd.read_excel(file_path, sheet_name="Anleihen", skiprows=2)

# Load historical indices data
df_etf = pd.read_excel(file_path, sheet_name="Indizes")

df_anleihen = df_anleihen.drop(df_anleihen.columns[[6, 11, 12, 13, 14, 15]], axis=1)
df_anleihen.columns = ["date", "Australien", "Kanada", "Frankreich", "Deutschland", "Japan", "Spanien", "Schweiz", "Großbritannien", "USA"]

df_etf.columns = ["date", "Dow Jones", "S&P-500", "S&P/TSX Composite", "CAC 40", "FTSE 100", "Bovespa", "Shanghai Composite", "Korea Composite", "NIKKEI 225", "IBEX 35", "S&P/ASX 50", "OMX Stockholm", "HANG SENG", "BSE SENSEX", "Dax", "FTSE All"]

df_etf.head(10)

,date,Dow Jones,S&P-500,S&P/TSX Composite,CAC 40,FTSE 100,Bovespa,Shanghai Composite,Korea Composite,NIKKEI 225,IBEX 35,S&P/ASX 50,OMX Stockholm,HANG SENG,BSE SENSEX,Dax,FTSE All
0,1955,19.445328,NaN,22.631374,NaN,NaN,NaN,NaN,NaN,19.545620,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1956,2.266585,NaN,5.306617,NaN,NaN,NaN,NaN,NaN,28.999977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1957,-12.769536,NaN,-23.516293,NaN,NaN,NaN,NaN,NaN,-14.133008,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1958,33.959926,NaN,26.754762,NaN,NaN,NaN,NaN,NaN,40.919984,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1959,16.398527,NaN,1.345578,NaN,NaN,NaN,NaN,NaN,31.663857,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1960,-9.342617,NaN,-1.864563,NaN,NaN,NaN,NaN,NaN,55.073839,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1961,18.712757,NaN,28.657708,NaN,NaN,NaN,NaN,NaN,5.593679,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1962,-10.810515,NaN,-10.253264,NaN,NaN,NaN,NaN,NaN,-0.849504,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1963,16.998927,NaN,11.720377,NaN,NaN,NaN,NaN,NaN,-13.146019,NaN,NaN,NaN,NaN,NaN,NaN,15.178760
9,1964,14.572384,12.969875,21.462623,NaN,NaN,NaN,NaN,NaN,-1.390127,NaN,NaN,NaN,NaN,NaN,NaN,-10.046836


In [ ]:
# Define the range of years
year_range = range(1, 31) 

# Initialize the structure for the JSON
json_structure = {
    "assets": []
}

# Function to process columns and add them to the JSON structure
def process_columns(df, category):
    for column_name in df.columns:
        if column_name == 'date':
            continue  

        growth_column = df[column_name]
        date_column = df['date']

        # Add a new asset entry for the current column
        asset_entry = {
            "category": category,
            "label": f"{column_name}",
            "kennung": f"{category} {column_name}",
            "id": f"{category} {column_name}",
            "data": [] 
        }

        # Iterate through all values of year
        for year in year_range:
            
            highest_value = float('-inf')
            lowest_value = float('inf')
            highest_year = None
            lowest_year = None
            highest_values_list = []  
            lowest_values_list = [] 
            median_values_list = []  
            values_with_years = [] 

            # Iterate through all rows where there are numbers
            valid_growths = growth_column.dropna()
            valid_dates = date_column[~growth_column.isna()]  

            for i in range(len(valid_growths) - year + 1): 
                value = 1
                growth_array = [] 
                for growth in valid_growths[i:i + year]:
                    value *= (1 + growth / 100)  
                    growth_array.append(round(growth, 2))  
                
                values_with_years.append((value, valid_dates.iloc[i], growth_array))  
                if value > highest_value:
                    highest_value = value
                    highest_year = valid_dates.iloc[i]
                    highest_values_list = growth_array  
                if value < lowest_value:
                    lowest_value = value
                    lowest_year = valid_dates.iloc[i]
                    lowest_values_list = growth_array  

            # Calculate the median value and find its corresponding year and array
            if values_with_years:  
                sorted_values = sorted(values_with_years, key=lambda v: v[0])
                median_index = len(sorted_values) // 2
                median_value, median_year, median_values_list = sorted_values[median_index]

                # Create dictionaries for max, median, and min arrays
                max_dict = {
                    "type": f"max",
                    "duration": f"{year}",
                    "year": str(highest_year),
                    "growth_array": highest_values_list
                }
                median_dict = {
                    "type": f"median",
                    "duration": f"{year}",
                    "year": str(median_year),
                    "growth_array": median_values_list
                }
                min_dict = {
                    "type": f"min",
                    "duration": f"{year}",
                    "year": str(lowest_year),
                    "growth_array": lowest_values_list
                }

                # Append the dictionaries to the "data" key in the asset entry
                asset_entry["data"].extend([max_dict, median_dict, min_dict])
            else:
                print(f"No valid values for column '{column_name}' and year = {year}. Skipping...")

        # Append the asset entry to the JSON structure
        json_structure["assets"].append(asset_entry)

# Process both `df_anleihen` and `df_etf`
process_columns(df_anleihen, "Anleihe")
process_columns(df_etf, "Index")

# Save the JSON structure to a file
with open("export/data.json", "w") as json_file:
    json.dump(json_structure, json_file, separators=(",", ":")) 

print("JSON file saved successfully!")

JSON file saved successfully!
